<a href="https://colab.research.google.com/github/koleshjr/Fake_News_Classification/blob/main/Fake_News_or_Real_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classification
* Can you recognize which articles are real and which one is fake? 

In [1]:
!pip install -q datasets
!pip install transformers
!pip install optuna
!pip install sentencepiece
!pip install evaluate


!pip install mlflow wandb



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.3 MB/s eta 0:00

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
from pathlib import Path
path = '/content/drive/MyDrive/fake_news_classifier/'

In [4]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import seaborn as sns 
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import mlflow
import os
import seaborn as sns
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import transformers 
import wandb

In [47]:
!wandb login 
wandb.init(project="classify_fake_or_real_news")

wandb: Currently logged in as: koleshjr (team-kolesh). Use `wandb login --relogin` to force relogin


In [6]:
%env WANDB_LOG_MODEL=true   #log every trained model

env: WANDB_LOG_MODEL=true   #log every trained model


In [7]:

transformers.set_seed(1) 

# ds_1 = pd.read_csv(path + 'FA-KES-Dataset.csv')
fake = pd.read_csv(path + 'Fake.csv')
true = pd.read_csv(path + 'True.csv')


fake['target'] =  0
true['target'] = 1
df = pd.concat([fake, true])
df.head()
     






,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [8]:
df.target.value_counts()


0    23481
1    21417
Name: target, dtype: int64

### Training

In [9]:
model_nm = "bert-base-uncased"

tokz = AutoTokenizer.from_pretrained(model_nm)



model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=2, hidden_dropout_prob=0.2)#, gradient_checkpointing = True).to(device)
# gradient_checkpointing: for saving memory - but will make computation slower

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
sep = tokz.sep_token
sep

'[SEP]'

In [11]:
# df['inputs'] = df.title + sep + df.text + sep + df.subject
df['inputs'] = df.title.str.lower() + sep + df.text.str.lower()
df['inputs'].head()

0     donald trump sends out embarrassing new year’...
1     drunk bragging trump staffer started russian ...
2     sheriff david clarke becomes an internet joke...
3     trump is so obsessed he even has obama’s name...
4     pope francis just called out donald trump dur...
Name: inputs, dtype: object

In [12]:
df = df.rename(columns = {'target':'label'})
df_train, df_temp = train_test_split(df, train_size=0.8, stratify = df['label'])
df_valid, df_test = train_test_split(df_temp, test_size=0.5,stratify = df_temp['label'])

df_train.shape, df_valid.shape, df_test.shape

((35918, 6), (4490, 6), (4490, 6))

In [13]:
adverse = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "valid": Dataset.from_pandas(df_valid),
    "test": Dataset.from_pandas(df_test)
    })

Tokenizing function

In [15]:
def tokenize(x): return tokz(x["inputs"],truncation=True, padding=True, max_length=512)



adverse_encoded = adverse.map(tokenize, batched=True, batch_size=8,remove_columns=('title', 'text', 'subject', 'date','__index_level_0__')) #, 


Map:   0%|          | 0/35918 [00:00<?, ? examples/s]

Map:   0%|          | 0/4490 [00:00<?, ? examples/s]

Map:   0%|          | 0/4490 [00:00<?, ? examples/s]

In [16]:
batch_size = 16
model_name = f"{model_nm}-finetuned-adverse"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=3,
                                  learning_rate= 4e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.03,
                                  evaluation_strategy="epoch",
                                logging_strategy="epoch",
                                  disable_tqdm=False,
                                  log_level="error",
                                 fp16 = True,  # for saving memory
                                 gradient_accumulation_steps=8, # for saving memory
                                  gradient_checkpointing=True, # for saving memory
                                 report_to="wandb")


In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    f1 = f1_score(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [18]:
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=adverse_encoded["train"],
                  eval_dataset=adverse_encoded["valid"],
                  tokenizer=tokz)
trainer.train();  



/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.021300,0.001227,0.999777,0.999767
1,0.000200,0.001621,0.999777,0.999767
2,0.000400,0.000541,0.999777,0.999767


In [19]:
trainer.evaluate()

{'eval_loss': 0.0005408995784819126,
 'eval_accuracy': 0.9997772828507795,
 'eval_f1': 0.999766518795237,
 'eval_runtime': 58.2473,
 'eval_samples_per_second': 77.085,
 'eval_steps_per_second': 9.649,
 'epoch': 3.0}

### Saving the Model and Loading it

In [23]:
# Save the model
trainer.save_model("/content/drive/MyDrive/fake_news_classifier/best_bert_model")



### Push the model to hugging face

In [26]:
from huggingface_hub import notebook_login
notebook_login()

In [31]:
trainer.push_to_hub()

#### loading the model for inference

In [32]:
# Load the model
loaded_model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/fake_news_classifier/best_bert_model",
    num_labels=2
)

# Load the tokenizer
loaded_tokenizer = AutoTokenizer.from_pretrained(
    "/content/drive/MyDrive/fake_news_classifier/best_bert_model",
)

# Max length
MAX_LENGTH = 512

In [36]:
example = df_test.inputs.iloc[0]

In [37]:


# Our example text to pass to our fine tuned model
text = example

def get_result(text, message=True):
    encoded_input = loaded_tokenizer(text, truncation=True, padding='max_length',
                                     max_length=MAX_LENGTH, return_tensors='pt')
    output = loaded_model(**encoded_input)
    result = output[0].detach().numpy()
    probs = torch.sigmoid(output[0]).detach().numpy()
    class_label = np.argmax(result)
    
    if message:
        print(f'The predicted class is label: {str(class_label)} with a probability of {probs[0][0]}')
    
    return result, class_label, probs


# Run your result through the function
result, class_label, probs = get_result(text)
     


The predicted class is label: 0 with a probability of 0.993893563747406


### Perform evaluation on the whole held_out test set

In [1]:
# Perform predictions on the test dataset
preds = trainer.predict(adverse_encoded["test"])



In [48]:
trainer.evaluate(adverse_encoded['test'])

{'eval_loss': 3.0081442673690617e-05,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_runtime': 64.0873,
 'eval_samples_per_second': 70.061,
 'eval_steps_per_second': 8.769,
 'epoch': 3.0}

Evaluation accuracy and f1 of 100% wow

In [49]:
wandb.finish()

eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,1.0
eval/f1,1.0
eval/loss,3e-05
